In [56]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import scipy.integrate as integrate

In [82]:
hazard_rates = pd.read_csv('hazard_rates.csv')
discount_factors = pd.read_csv('discount_factors.csv')
discount_factors = discount_factors.set_index('dates')
dfc = discount_factors.columns
discount_factors['zero'] = 1
discount_factors = discount_factors[np.append('zero', dfc )]
coupons = pd.read_excel('treasury_bond_note_coupons.xlsx')
cds = pd.read_csv('cds_data_processed.csv')
discount_factors.reset_index(inplace=True)

In [83]:
mean_coupons = coupons.groupby('Tenor').mean().reset_index()

In [84]:
cds['Open_6m'] = cds['Open_6m']/100
cds['Open_1yr'] = cds['Open_1yr']/100
cds['Open_2yr'] = mean_coupons[mean_coupons['Tenor'] == 2]['Cpn'].values[0] - cds['Open_2yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 3]['Cpn'].values[0] - cds['Open_3yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 5]['Cpn'].values[0] - cds['Open_5yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 7]['Cpn'].values[0] - cds['Open_7yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 10]['Cpn'].values[0] - cds['Open_10yr']/100

In [85]:
time_periods_m = np.array([0, 1/12,3/12, 6/12, 9/12, 1])
time_periods_y = np.arange(2, 31, 1)
time_periods = np.append(time_periods_m, time_periods_y)


In [86]:
discount_factors.head()

,dates,zero,1m,3m,6m,9m,12m,2y,3y,4y,...,21y,22y,23y,24y,25y,26y,27y,28y,29y,30y
0,2008-09-19,1,0.998385,0.995086,0.990174,0.984897,0.979345,0.951522,0.917024,0.880339,...,0.402284,0.386413,0.371267,0.356599,0.342169,0.327700,0.312925,0.297590,0.281410,0.264118
1,2008-09-25,1,0.998427,0.995413,0.990934,0.986135,0.980911,0.951436,0.914465,0.877125,...,0.402427,0.388342,0.374797,0.361392,0.347695,0.333278,0.317701,0.300480,0.281210,0.259418
2,2008-09-29,1,0.998616,0.996014,0.992124,0.988072,0.983554,0.959236,0.924501,0.888527,...,0.421357,0.405718,0.390869,0.376550,0.362489,0.348419,0.334048,0.319115,0.303289,0.286308
3,2008-09-30,1,0.998512,0.995585,0.991234,0.986540,0.981238,0.953419,0.917388,0.879978,...,0.407647,0.393056,0.379102,0.365474,0.351804,0.337713,0.322826,0.306770,0.289167,0.269649
4,2008-10-01,1,0.998562,0.995818,0.991765,0.987373,0.982202,0.954446,0.919233,0.882789,...,0.418106,0.403817,0.390128,0.376678,0.363109,0.349048,0.334131,0.317959,0.300130,0.280299


In [87]:
discount_factors_funcs = discount_factors.apply(lambda row: interp1d(time_periods, row[1:], kind='cubic'), axis = 1)

(3356,)

In [90]:

integrate.quad(lambda x: discount_factors_funcs[0](x), 0, 0.5)[0]

0.4975495519485934